# Using Time Series to more accurately predict wave heights

In [2]:
import utilities.misc as misc
import utilities.metrics as mtr
import utilities.ml_functions as ml
import pandas as pd

In [12]:
# convert datetime to unix
df = pd.read_csv('clean_weather/valid_dates/m2_valid_dates.csv', parse_dates=['date', 'time'])
start_date = min(df['time'])
end_date = max(df['time'])

df['timestamps'] = pd.to_datetime(df['time'])
df['timestamps'] = (df['timestamps'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')  

df.head()


,Unnamed: 0,time,station_id,AtmosphericPressure,WindDirection,WindSpeed,Gust,WaveHeight,WavePeriod,MeanWaveDirection,Hmax,SeaTemperature,date,timestamps
0,1953,2021-11-29 00:00:00,M2,1020.0,330.0,2.92,3.9,0.6,4.0,NaN,NaN,8.6,2001-05-05,1638144000
1,1955,2021-11-29 01:00:00,M2,1020.8,330.0,4.09,5.8,0.4,4.0,NaN,NaN,8.6,2001-05-05,1638147600
2,1957,2021-11-29 02:00:00,M2,1020.8,350.0,4.09,5.8,0.4,4.0,NaN,NaN,8.7,2001-05-05,1638151200
3,1959,2021-11-29 03:00:00,M2,1021.2,350.0,1.95,5.8,0.4,4.0,NaN,NaN,8.7,2001-05-05,1638154800
4,1961,2021-11-29 04:00:00,M2,1021.4,30.0,2.92,5.8,0.4,4.0,NaN,NaN,8.7,2001-05-05,1638158400


In [5]:
dates = pd.to_datetime(['2019-01-15 13:30:00'])
(dates - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')  
# Int64Index([1547559000], dtype='int64')

Int64Index([1547559000], dtype='int64')

In [ ]:
# plot data in time 


In [ ]:
# predict using daily seasonality